# 📊 Lab 11: Processing Raw Data with LLMs
## Module 11 - Data Cleaning, Preprocessing & Synthetic Generation

**Duration:** 25 minutes

**Objectives:**
- Clean and preprocess banking data
- Mask PII for compliance
- Generate synthetic training data

**Banking Scenario:** Prepare loan application data for model training

---

In [1]:
import os
import json
import re

# =============================================================================
# GOOGLE COLAB SETUP - Add these secrets (click 🔑 icon):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: https://xxx.openai.azure.com/
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing")
except Exception:
    print("⚠️ Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Client ready")

✅ Credentials loaded. Model: gpt-4o
✅ Client ready


## Part 1: PII Masking

In [2]:
def mask_pii(text: str) -> str:
    """Mask common PII patterns in banking data"""
    # SSN: XXX-XX-XXXX
    text = re.sub(r'\b\d{3}-\d{2}-\d{4}\b', '[SSN]', text)
    # Account numbers: 10-16 digits
    text = re.sub(r'\b\d{10,16}\b', '[ACCOUNT]', text)
    # Phone: various formats
    text = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '[PHONE]', text)
    # Email
    text = re.sub(r'\b[\w.-]+@[\w.-]+\.\w+\b', '[EMAIL]', text)
    # Names (simple pattern)
    text = re.sub(r'\b(Mr\.|Mrs\.|Ms\.)\s+[A-Z][a-z]+\s+[A-Z][a-z]+\b', '[NAME]', text)
    return text

# Test PII masking
raw_text = """Customer John Smith (SSN: 123-45-6789) applied for a loan.
Account: 1234567890123456, Phone: 555-123-4567
Email: john.smith@email.com"""

masked = mask_pii(raw_text)
print("Original:")
print(raw_text)
print("\nMasked:")
print(masked)

Original:
Customer John Smith (SSN: 123-45-6789) applied for a loan.
Account: 1234567890123456, Phone: 555-123-4567
Email: john.smith@email.com

Masked:
Customer John Smith (SSN: [SSN]) applied for a loan.
Account: [ACCOUNT], Phone: [PHONE]
Email: [EMAIL]


## Part 2: Data Cleaning Pipeline

In [5]:
def clean_loan_document(raw_doc: str) -> dict:
    """Clean and structure a raw loan document"""
    # Step 1: Mask PII
    masked = mask_pii(raw_doc)

    # Step 2: Use LLM to extract structured data
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{
            "role": "system",
            "content": "Extract loan details from the document. Return JSON with: loan_type, amount, rate, term_years, applicant_income, credit_score"
        }, {
            "role": "user",
            "content": masked
        }],
        response_format={"type": "json_object"}
    )

    extracted = json.loads(response.choices[0].message.content)
    extracted["_masked_text"] = masked
    return extracted

# Test cleaning
raw_loan = """LOAN APPLICATION
Applicant: Mr. John Smith
SSN: 123-45-6789
Loan Type: Mortgage
Requested Amount: $350,000
Interest Rate: 6.25%
Term: 30 years
Annual Income: $95,000
Credit Score: 720"""

cleaned = clean_loan_document(raw_loan)
print("Cleaned & Structured:")
print(json.dumps(cleaned, indent=2))

Cleaned & Structured:
{
  "loan_type": "Mortgage",
  "amount": 350000,
  "rate": 6.25,
  "term_years": 30,
  "applicant_income": 95000,
  "credit_score": 720,
  "_masked_text": "LOAN APPLICATION\nApplicant: [NAME]\nSSN: [SSN]\nLoan Type: Mortgage\nRequested Amount: $350,000\nInterest Rate: 6.25%\nTerm: 30 years\nAnnual Income: $95,000\nCredit Score: 720"
}


## Part 3: Synthetic Data Generation

In [6]:
def generate_synthetic_loans(base_example: dict, count: int = 5) -> list:
    """Generate synthetic loan applications"""
    prompt = f"""Generate {count} realistic loan application variations.

Base example:
{json.dumps(base_example, indent=2)}

Requirements:
- Vary income ($40K-$300K)
- Vary credit scores (580-850)
- Include different loan types (mortgage, auto, personal)
- Keep realistic correlations
- Return as JSON array with key "applications"
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )

    return json.loads(response.choices[0].message.content).get("applications", [])

# Generate synthetic data
base = {"loan_type": "mortgage", "amount": 250000, "rate": 6.5, "income": 85000, "credit_score": 720}
synthetic = generate_synthetic_loans(base, count=5)

print(f"Generated {len(synthetic)} synthetic applications:")
for i, app in enumerate(synthetic):
    print(f"  {i+1}. {app.get('loan_type', 'N/A')}: ${app.get('amount', 0):,} @ {app.get('rate', 0)}%")

Generated 5 synthetic applications:
  1. mortgage: $350,000 @ 7.2%
  2. auto: $30,000 @ 4.5%
  3. personal: $20,000 @ 10.5%
  4. mortgage: $500,000 @ 6.9%
  5. auto: $18,000 @ 5.0%


## Part 4: Prepare JSONL for Fine-Tuning

In [7]:
def create_training_example(loan_data: dict) -> dict:
    """Convert loan data to fine-tuning format"""
    return {
        "messages": [
            {"role": "system", "content": "Extract loan details as JSON."},
            {"role": "user", "content": f"Process this loan: {loan_data.get('_masked_text', str(loan_data))}"},
            {"role": "assistant", "content": json.dumps({k: v for k, v in loan_data.items() if not k.startswith('_')})}
        ]
    }

# Create training file
training_data = [create_training_example(app) for app in synthetic]

print("Training data sample:")
print(json.dumps(training_data[0], indent=2))

Training data sample:
{
  "messages": [
    {
      "role": "system",
      "content": "Extract loan details as JSON."
    },
    {
      "role": "user",
      "content": "Process this loan: {'loan_type': 'mortgage', 'amount': 350000, 'rate': 7.2, 'income': 125000, 'credit_score': 760}"
    },
    {
      "role": "assistant",
      "content": "{\"loan_type\": \"mortgage\", \"amount\": 350000, \"rate\": 7.2, \"income\": 125000, \"credit_score\": 760}"
    }
  ]
}


---
## ✅ Lab 11 Complete!

**Key Takeaways:**
- Always mask PII before processing
- LLMs can extract structured data from raw text
- Synthetic data helps with limited real data
- JSONL format is required for fine-tuning